<a href="https://colab.research.google.com/github/assuahborsah/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,526 kB]
Hit:13 http://ppa.launchpa

In [3]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [4]:
# Start a SparkSession
import findspark
findspark.init()

In [5]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-17 03:05:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-07-17 03:05:05 (6.52 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
# Load Amazon Data into Spark DataFrame

In [8]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [9]:
# Create DataFrames to match tables

In [10]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [30]:
# Create the customers_table DataFrame
customers_table_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_table_df = customers_table_df.dropDuplicates(["customer_id"])
customers_table_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43679767|             1|
|   32024654|             1|
|   52913169|             1|
|   24297214|             1|
|   26096454|             1|
|   38247118|             1|
|   44009906|             1|
|    1753876|             1|
|   26195644|             1|
|   23042837|             1|
|   11487525|             2|
|   28258386|             1|
|   42560427|             2|
|   24540309|             1|
|    1967239|             1|
|   14217455|             1|
|   20289484|            26|
|   47103434|             1|
|   28638887|             1|
|     740678|             1|
+-----------+--------------+
only showing top 20 rows



In [12]:
customers_table_df.count()

1516428

In [28]:
# Create the products_table DataFrame and drop duplicates adding 'product_id' as the parameter. 
products_table_df = df.select(["product_id","product_title"]).drop_duplicates(['product_id'])
products_table_df = products_table_df.dropDuplicates(["product_id"])
products_table_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|6040161299|Santa Cruz Scream...|
|B0000AXTWB|Kwik Tek SF-1C Su...|
|B0000BYDQX|Bellows Foot Pump...|
|B0000DYNXN|DIN &amp; Yoke Ta...|
|B0000E66XJ|Black Diamond Zio...|
|B00012O8B4|Razor E100 Electr...|
|B00029NCOK|ATV Logic ATVHP-B...|
|B0002BJZ6W|Pro XL-C Corragat...|
|B0002CTKO8|Heat Factory Flee...|
|B0002LKM8W|Weekender Men's T...|
|B0002M9DO0|STABILicers Maxx ...|
|B000650ZMW|Panaracer, 700x32...|
|B00066Z29W|             Fairing|
|B00068NHMO|Marmot Men's Midw...|
|B0006FYKCI|Transpack Edge Is...|
|B0006UOKFA|FireOne Windproof...|
|B0006VSTOW|Flowlab Blue Logo...|
|B0007IS6BA|Columbia Bugaboo ...|
|B0007QCO4M|              Bantam|
|B0007QCOP6|Victorinox Swiss ...|
+----------+--------------------+
only showing top 20 rows



In [14]:
products_table_df.count()

391729

In [15]:
# Create the review_id_table DataFrame. 
#Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")

review_id_table_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table_df = review_id_table_df.dropDuplicates(["review_id"])

review_id_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1007GK29ILBBG|   33535377|B00A8M7KNK|     897619678| 2014-09-26|
|R100XGCKXIBW11|   48559865|B0083ADVA0|      29148454| 2014-10-21|
| R10101XZBVPWP|   33118725|B000P9GQNG|     962193881| 2014-10-21|
|R101LSOWRLO69P|   14501114|B00BJAPMSK|     408508117| 2015-03-27|
|R1026JNJYK9U4F|   51334978|B00H5MQKXW|     294414264| 2015-05-18|
|R102ARPFRF7LNN|   47645554|B000Z89JFO|     350070285| 2008-07-22|
|R102OS6NE9G01O|   17227287|B003FMCSZA|     627817293| 2015-06-30|
|R102RPH47F9E62|   13892552|B000TQB7VG|     781855672| 2015-03-19|
|R103JA3YKTCR8G|   45463076|B000AO9PCU|      21674941| 2014-09-29|
|R103NCNSLV2ABU|   14805208|B002DX1DWG|     909595981| 2014-07-27|
|R1044WRUKY3TBV|    1110424|B000TFRHXE|     121250191| 2015-08-26|
|R1048PMOVIYRKZ|   17295140|B0055P25QW|     805929113| 2013-04

In [16]:
review_id_table_df.count()

2302401

In [17]:
# drop null values
new_review_id_table_df = review_id_table_df.dropna()
new_review_id_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1007GK29ILBBG|   33535377|B00A8M7KNK|     897619678| 2014-09-26|
|R100XGCKXIBW11|   48559865|B0083ADVA0|      29148454| 2014-10-21|
| R10101XZBVPWP|   33118725|B000P9GQNG|     962193881| 2014-10-21|
|R101LSOWRLO69P|   14501114|B00BJAPMSK|     408508117| 2015-03-27|
|R1026JNJYK9U4F|   51334978|B00H5MQKXW|     294414264| 2015-05-18|
|R102ARPFRF7LNN|   47645554|B000Z89JFO|     350070285| 2008-07-22|
|R102OS6NE9G01O|   17227287|B003FMCSZA|     627817293| 2015-06-30|
|R102RPH47F9E62|   13892552|B000TQB7VG|     781855672| 2015-03-19|
|R103JA3YKTCR8G|   45463076|B000AO9PCU|      21674941| 2014-09-29|
|R103NCNSLV2ABU|   14805208|B002DX1DWG|     909595981| 2014-07-27|
|R1044WRUKY3TBV|    1110424|B000TFRHXE|     121250191| 2015-08-26|
|R1048PMOVIYRKZ|   17295140|B0055P25QW|     805929113| 2013-04

In [18]:
# count number of rows after dropping null values.

new_review_id_table_df.count()

2302303

In [32]:
# Create the vine_table. DataFrame
vine_table_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_table_df = vine_table_df.dropDuplicates(["review_id"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1007GK29ILBBG|          5|            3|          3|   N|                Y|
|R100XGCKXIBW11|          5|            1|          1|   N|                N|
| R10101XZBVPWP|          5|            1|          1|   N|                Y|
|R101LSOWRLO69P|          5|            0|          0|   N|                Y|
|R1026JNJYK9U4F|          5|            0|          0|   N|                Y|
|R102ARPFRF7LNN|          4|           30|         32|   N|                Y|
|R102OS6NE9G01O|          4|            0|          0|   N|                Y|
|R102RPH47F9E62|          5|            0|          0|   N|                Y|
|R103JA3YKTCR8G|          5|            1|          2|   N|                Y|
|R103NCNSLV2ABU|          5|            2|          2|   N|     

In [20]:
vine_table_df.count()

2302401

In [21]:
# drop null values
new_vine_table_df = vine_table_df.dropna()
new_vine_table_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [22]:
# count rows after dropping the null values.

new_vine_table_df.count()

2302392

In [23]:
# Connect to the AWS RDS instance and write each DataFrame to its table. 

In [24]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')

mode = "append"
jdbc_url="jdbc:postgresql://database-1.ce2d8g4atlfe.us-east-1.rds.amazonaws.com:5432/challenge_schema"
config = {"user":"postgres", 
          "password":password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [26]:
# Write review_id_df to table in RDS
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [29]:
# Write products_df to table in RDS
# about 3 min
products_table_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [31]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_table_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [33]:
# Write vine_df to table in RDS
# 11 minutes
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)